In [1]:
import numpy as np # For arithmetics and arrays
import math # For inbuilt math functions
import pandas as pd # For handling data frames
import collections # used for dictionaries and counters
from itertools import permutations # used to find permutations

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function to easily split data into training and testing samples
from sklearn.decomposition import PCA # Principal component analysis used to reduce the number of features in a model
from sklearn.preprocessing import StandardScaler # used to scale data to be used in the model
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

import pickle 
import seaborn as sns 
sns.set(style="ticks")
import matplotlib.pyplot as plt

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

In [2]:
df = pd.read_csv("lang_data.csv")

In [3]:
df = df.dropna()

In [4]:
df['text'] = df['text'].astype(str)

In [5]:
df['language'] = df['language'].astype(str)

In [6]:
#feature creation
punc = ('!', "," ,"\'" ,";" ,"\"", ".", "-" ,"?")
vowels=['a','e','i','o','u']

In [7]:
same_consecutive_vowels = ['aa','ee', 'ii', 'oo', 'uu'] 
consecutive_vowels = [''.join(p) for p in permutations(vowels,2)]
dutch_combos = ['ij']

In [8]:
df['word_count'] = df['text'].apply(lambda x : len(x.split()))
df['character_count'] = df['text'].apply(lambda x : len(x.replace(" ","")))
df['word_density'] = df['word_count'] / (df['character_count'] + 1)
df['punc_count'] = df['text'].apply(lambda x : len([a for a in x if a in punc]))
df['v_char_count'] = df['text'].apply(lambda x : len([a for a in x if a.casefold() == 'v']))
df['w_char_count'] = df['text'].apply(lambda x : len([a for a in x if a.casefold() == 'w']))
df['ij_char_count'] = df['text'].apply(lambda x : sum([any(d_c in a for d_c in dutch_combos) for a in x.split()]))
df['num_double_consec_vowels'] = df['text'].apply(lambda x : sum([any(c_v in a for c_v in same_consecutive_vowels) for a in x.split()]))
df['num_consec_vowels'] = df['text'].apply(lambda x : sum([any(c_v in a for c_v in consecutive_vowels) for a in x.split()]))
df['num_vowels'] = df['text'].apply(lambda x : sum([any(v in a for v in vowels) for a in x.split()]))
df['vowel_density'] = df['num_vowels']/df['word_count']
df['capitals'] = df['text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
df['caps_vs_length'] = df.apply(lambda row: float(row['capitals'])/float(row['character_count']),axis=1)
df['num_exclamation_marks'] =df['text'].apply(lambda x: x.count('!'))
df['num_question_marks'] = df['text'].apply(lambda x: x.count('?'))
df['num_punctuation'] = df['text'].apply(lambda x: sum(x.count(w) for w in punc))
df['num_unique_words'] = df['text'].apply(lambda x: len(set(w for w in x.split())))
df['num_repeated_words'] = df['text'].apply(lambda x: len([w for w in collections.Counter(x.split()).values() if w > 1]))
df['words_vs_unique'] = df['num_unique_words'] / df['word_count']
df['encode_ascii'] = np.nan

In [9]:
for i in range(len(df)):
    try:
        df['text'].iloc[i].encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        df['encode_ascii'].iloc[i] = 0
    else:
        df['encode_ascii'].iloc[i] = 1

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
df.groupby('language').mean().T

language,Afrikaans,English,Nederlands
word_count,10.503912,4.072506,5.746269
character_count,43.541471,16.841849,26.074627
word_density,0.234060,0.226490,0.209378
punc_count,1.507042,0.317275,1.223881
v_char_count,0.652582,0.126521,0.358209
w_char_count,0.904538,0.291971,0.522388
ij_char_count,0.000000,0.000000,0.268657
num_double_consec_vowels,1.696401,0.178589,1.014925
num_consec_vowels,2.773083,0.536253,1.134328
num_vowels,9.114241,3.742579,5.671642


In [11]:
df.corr(method ='pearson')

,word_count,character_count,word_density,punc_count,v_char_count,w_char_count,ij_char_count,num_double_consec_vowels,num_consec_vowels,num_vowels,vowel_density,capitals,caps_vs_length,num_exclamation_marks,num_question_marks,num_punctuation,num_unique_words,num_repeated_words,words_vs_unique,encode_ascii
word_count,1.000000,0.963818,0.284142,0.656144,0.499937,0.576566,0.002617,0.714460,0.769932,0.985911,-0.158172,0.408048,-0.449626,0.161356,0.054530,0.656144,0.985286,0.785662,-0.609634,-0.500432
character_count,0.963818,1.000000,0.066516,0.685772,0.535066,0.579176,0.012066,0.737948,0.801404,0.960918,-0.089117,0.409393,-0.486624,0.174488,0.065903,0.685772,0.950250,0.751124,-0.570829,-0.506098
word_density,0.284142,0.066516,1.000000,0.001107,0.002239,0.061311,-0.032624,0.050620,0.042554,0.249241,-0.356830,0.081018,0.062398,-0.011030,0.001162,0.001107,0.306803,0.143736,-0.178659,-0.077037
punc_count,0.656144,0.685772,0.001107,1.000000,0.370564,0.374964,0.042030,0.548477,0.563451,0.621920,-0.228459,0.385474,-0.287332,0.205043,0.119390,1.000000,0.652952,0.487491,-0.379424,-0.392489
v_char_count,0.499937,0.535066,0.002239,0.370564,1.000000,0.233034,0.022992,0.389565,0.424615,0.499093,-0.038331,0.250590,-0.249199,0.095870,0.025231,0.370564,0.502898,0.367544,-0.287014,-0.337758
w_char_count,0.576566,0.579176,0.061311,0.374964,0.233034,1.000000,0.005225,0.468219,0.413632,0.580779,-0.002004,0.204286,-0.280512,0.111205,0.101982,0.374964,0.562223,0.464511,-0.353433,-0.298730
ij_char_count,0.002617,0.012066,-0.032624,0.042030,0.022992,0.005225,1.000000,0.000365,-0.010899,0.012548,0.047971,-0.021422,-0.048555,-0.007353,-0.007675,0.042030,0.007278,-0.016064,0.017089,0.023861
num_double_consec_vowels,0.714460,0.737948,0.050620,0.548477,0.389565,0.468219,0.000365,1.000000,0.588212,0.707235,-0.079424,0.245026,-0.352140,0.174573,0.040402,0.548477,0.705972,0.563045,-0.412084,-0.471362
num_consec_vowels,0.769932,0.801404,0.042554,0.563451,0.424615,0.413632,-0.010899,0.588212,1.000000,0.776344,-0.044574,0.286679,-0.401469,0.121268,0.033055,0.563451,0.759246,0.604320,-0.452598,-0.436547
num_vowels,0.985911,0.960918,0.249241,0.621920,0.499093,0.580779,0.012548,0.707235,0.776344,1.000000,-0.022570,0.357889,-0.480243,0.158128,0.057858,0.621920,0.968959,0.782950,-0.614507,-0.450493


In [12]:
#split dataset
feature_cols = list(df.columns)[2:]
X = df[feature_cols] #features
y = df[['language']]  #target

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% train and 20% test

In [14]:
#standardize data
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
pca = PCA(.95)
pca.fit(X_train)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [15]:
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [17]:
dt_clf = DecisionTreeClassifier()
dt_clf = dt_clf.fit(X_train,y_train) 

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/tree/tree.py:163: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)


In [18]:
pkl_file = "decision_tree_model.pkl"
with open(pkl_file, 'wb') as file:
    pickle.dump(dt_clf, file)
with open(pkl_file, 'rb') as file:
    dt_clf = pickle.load(file)
y_pred = dt_clf.predict(X_test)

In [19]:
accuracy_score_dt = accuracy_score(y_test, y_pred)

In [21]:
accuracy_score_dt

0.8806509945750453